In [1]:
import pandas as pd
import random
import numpy as np
from mlxtend.frequent_patterns import apriori,fpgrowth , association_rules
import time
import csv
import itertools
from mlxtend.preprocessing import TransactionEncoder

In [2]:
dataset1=pd.read_csv("Amazon.csv")


In [3]:
dataset2=pd.read_csv("Bestbuy.csv")


In [4]:
dataset3=pd.read_csv("K-Mart.csv")

In [5]:
dataset4=pd.read_csv("Nike.csv")


In [6]:
dataset5=pd.read_csv("Walmart.csv")


In [7]:
print("Select any one dataset from below list to generate Association rules")
print("1. Amazon Dataset")
print("2. BestBuy Dataset")
print("3. K-Mart Dataset")
print("4. Nike Dataset")
print("5. Walmart Dataset")

choice=int(input("Enter the dataset number(1-5): "))

if(choice<1 or choice>5):
    print("Invalid entry. Please enter a number between 1-5.")
print(choice)


Select any one dataset from below list to generate Association rules
1. Amazon Dataset
2. BestBuy Dataset
3. K-Mart Dataset
4. Nike Dataset
5. Walmart Dataset
Enter the dataset number(1-5): 1
1


In [8]:
def isInList(aList, item):
    if item in aList:
        return 1
    else:
        return -1  
    
def removeEmptyValues(aList, symbolToRemove):
    while(True):
        val = isInList(aList, symbolToRemove)
        if(val == 1):
            aList.remove(symbolToRemove)    
        else:
            break

def generateListOfListsFromCSV(fullFileName):

    with open(fullFileName, newline='', encoding='utf-8-sig') as f:
        reader = csv.reader(f, delimiter=",")
        data = list(reader)

    listOfLists = []

    for i in range(len(data)):
        removeEmptyValues(data[i], '')
        listOfLists.append(data[i])
        
    return listOfLists

if(choice==1):
    listoflist_dataset=generateListOfListsFromCSV("Amazon.csv")
if(choice==2):
    listoflist_dataset=generateListOfListsFromCSV("Bestbuy.csv")
if(choice==3):
    listoflist_dataset=generateListOfListsFromCSV("K-Mart.csv")
if(choice==4):
    listoflist_dataset=generateListOfListsFromCSV("Nike.csv")
if(choice==5):
    listoflist_dataset=generateListOfListsFromCSV("Walmart.csv")

# Brute Force Approach

In [9]:


brute_force_time=[]
#   This function takes all the frequent sets as the input and generates Association Rules
def create_ruleof_associ(freqSet): 
    associationRule = []
    for item in freqSet:
        if isinstance(item, list):
            if len(item) != 0:
                length = len(item) - 1
                while length > 0:
                    combinations = list(itertools.combinations(item, length))
                    temp = []
                    LHS = []
                    for RHS in combinations:
                        LHS = set(item) - set(RHS)
                        temp.append(list(LHS))
                        temp.append(list(RHS))
                        associationRule.append(temp)
                        temp = []
                    length = length - 1
    
    return associationRule


#   This function creates Frequent item sets by taking candidate sets as input
#   At the end, this function calls generateCandidatSets by feeding the output of the
#   current function as the input of the other function
def freqnitemset(listofcandidates, transactions, min_support, dataset, superfreqntlist): 
    list_freqntitems = []
    for i in range(len(listofcandidates)):
        if i%2 != 0:
            support = (listofcandidates[i] * 1.0 / transactions) * 100
            if support >= min_support:
                list_freqntitems.append(listofcandidates[i-1])
                list_freqntitems.append(listofcandidates[i])
            else:
                listofeliminateditems.append(listofcandidates[i-1])

    for k in list_freqntitems:
        superfreqntlist.append(k)

    if len(list_freqntitems) == 2 or len(list_freqntitems) == 0:
        #print("This will be returned")
        returnArray = superfreqntlist
        return returnArray

    else:
        findlistofcandidates(dataset, listofeliminateditems, list_freqntitems, transactions, min_support)
    
    
#   This function creates the final output of the algorithm by taking Association Rules as the input
def bruteforce(rules, dataset, min_support, minconfi):
    
    brute_op = []
    for rule in rules:
        supprtx = 0
        supprtxperc = 0
        supprtxny = 0
        supprtxnyperc = 0
        for transaction in dataset:
            if set(rule[0]).issubset(set(transaction)):
                supprtx = supprtx + 1
            if set(rule[0] + rule[1]).issubset(set(transaction)):
                supprtxny = supprtxny + 1
        supprtxperc = (supprtx * 1.0 / transactions) * 100
        supprtxnyperc = (supprtxny * 1.0 / transactions) * 100
        confidence = (supprtxnyperc / supprtxperc) * 100
        if confidence >= minconfi:
            supprtxstring = "Support Of X: " + str(round(supprtxperc, 2))
            supprtxnystr = "Support of X & Y: " + str(round(supprtxnyperc))
            confstr = "Confidence: " + str(round(confidence))

            brute_op.append(supprtxstring)
            brute_op.append(supprtxnystr)
            brute_op.append(confstr)
            brute_op.append(rule)
            
    
    return brute_op


#   This function creates Candidate sets by taking frequent sets as the input
#   At the end, this function calls freqnitemsets by feeding the output of the
#   crrent function as the input of the other function
def findlistofcandidates(dataset, listofeliminateditems, list_freqntitems, transactions, min_support):
    
    elementslist = []
    listaftercomb = []
    listofcandidateset = []
    for i in range(len(list_freqntitems)):
        if i%2 == 0:
            elementslist.append(list_freqntitems[i])
    for item in elementslist:
        templistcomb = []
        k = elementslist.index(item)
        for i in range(k + 1, len(elementslist)):
            for j in item:
                if j not in templistcomb:
                    templistcomb.append(j)
            for m in elementslist[i]:
                if m not in templistcomb:
                    templistcomb.append(m)
            listaftercomb.append(templistcomb)
            templistcomb = []
    sortcomblist = []
    distcomblist = []
    for i in listaftercomb:
        sortcomblist.append(sorted(i))
    for i in sortcomblist:
        if i not in distcomblist:
            distcomblist.append(i)
    listaftercomb = distcomblist
    for item in listaftercomb:
        count = 0
        for transaction in dataset:
            if set(item).issubset(set(transaction)):
                count = count + 1
        if count != 0:
            listofcandidateset.append(item)
            listofcandidateset.append(count)
    freqnitemset(listofcandidateset, transactions, min_support, dataset, superfreqntlist)


#   This function generates the first candidate set using the dataset
def createfirstcadidate(dataset):
    
    proddict = {}
    returnlist = []
    for data in dataset:
        for product in data:
            if product not in proddict:
               proddict[product] = 1
            else:
                 proddict[product] = proddict[product] + 1
    for key in proddict:
        templist = []
        templist.append(key)
        returnlist.append(templist)
        returnlist.append(proddict[key])
        templist = []
    return returnlist

min_support = float(input('Enter minimum Support: '))
minconfi = float(input('Enter minimum Confidence: '))
print("\n") 



min_support = int(min_support)
minconfi = int(minconfi)

nonFrequentSets = []
allFrequentItemSets = []
tempFrequentItemSets = []
dataset = []
listofeliminateditems = []
transactions = 0
superfreqntlist = []
something = 0


dataset=listoflist_dataset 

transactions = len(dataset)

firstCandidateSet = createfirstcadidate(dataset)

frequentItemSet = freqnitemset(firstCandidateSet, transactions, min_support, dataset, superfreqntlist)

associationRules = create_ruleof_associ(superfreqntlist)

FinalOp = bruteforce(associationRules, dataset, min_support, minconfi)

print("The input transactions for this dataset are: ",'\n')
for i in dataset:
    print(i)
print('\n')
#print(time)
counter = 1
if len(FinalOp) == 0:
    print("There are no association rules for this support and confidence. Please try to enter a lower support or lower confidence value")
else:
    print("Association Rules: ")
    for i in FinalOp:
        if counter == 4:
            print(str(i[0]) + "------>" + str(i[1]))
            counter = 0
        else:
            print(i, end='  ')
        counter = counter + 1

Enter minimum Support: 30
Enter minimum Confidence: 40


The input transactions for this dataset are:  

["A Beginner's Guide", ' Java: The Complete Reference', ' Java For Dummies', ' Android Programming: The Big Nerd Ranch']
["A Beginner's Guide", ' Java: The Complete Reference', ' Java For Dummies']
["A Beginner's Guide", ' Java: The Complete Reference', ' Java For Dummies', ' Android Programming: The Big Nerd Ranch', ' Head First Java 2nd Edition']
['Android Programming: The Big Nerd Ranch', ' Head First Java 2nd Edition ', ' Beginning Programming with Java']
['Android Programming: The Big Nerd Ranch', ' Beginning Programming with Java', ' Java 8 Pocket Guide']
["A Beginner's Guide", ' Android Programming: The Big Nerd Ranch', ' Head First Java 2nd Edition']
["A Beginner's Guide", ' Head First Java 2nd Edition ', ' Beginning Programming with Java']
['Java: The Complete Reference', ' Java For Dummies', ' Android Programming: The Big Nerd Ranch']
['Java For Dummies', ' Android Program

# Apriori Implementation from Python Libraries

In [10]:


min_support = float(input('Enter minimum Support: '))
minconfi = float(input('Enter minimum Confidence: '))

te = TransactionEncoder()
te_ary = te.fit_transform(listoflist_dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

# Use Apriori algorithm
frequent_itemsets = apriori(df, min_support=min_support, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=minconfi)

#Transactions for this input:
print("The input transactions for this dataset are: ",'\n')
for i in listoflist_dataset:
    print(i)
print('\n')
    
# Print results
print("Frequent Itemsets:")
print(frequent_itemsets)

#Association Rules:
print("\nAssociation Rules:")
print(rules)

Enter minimum Support: 0.3
Enter minimum Confidence: 0.7
The input transactions for this dataset are:  

["A Beginner's Guide", ' Java: The Complete Reference', ' Java For Dummies', ' Android Programming: The Big Nerd Ranch']
["A Beginner's Guide", ' Java: The Complete Reference', ' Java For Dummies']
["A Beginner's Guide", ' Java: The Complete Reference', ' Java For Dummies', ' Android Programming: The Big Nerd Ranch', ' Head First Java 2nd Edition']
['Android Programming: The Big Nerd Ranch', ' Head First Java 2nd Edition ', ' Beginning Programming with Java']
['Android Programming: The Big Nerd Ranch', ' Beginning Programming with Java', ' Java 8 Pocket Guide']
["A Beginner's Guide", ' Android Programming: The Big Nerd Ranch', ' Head First Java 2nd Edition']
["A Beginner's Guide", ' Head First Java 2nd Edition ', ' Beginning Programming with Java']
['Java: The Complete Reference', ' Java For Dummies', ' Android Programming: The Big Nerd Ranch']
['Java For Dummies', ' Android Program

# FP Growth Implementation from Python Libraries

In [12]:
te = TransactionEncoder()
te_ary = te.fit_transform(listoflist_dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

min_support = float(input('Enter minimum Support: '))
minconfi = float(input('Enter minimum Confidence: '))

# Use FP-growth algorithm
frequent_itemsets_fp = fpgrowth(df, min_support=min_support, use_colnames=True)

# Generate association rules
rules_fp = association_rules(frequent_itemsets_fp, metric="confidence", min_threshold=minconfi)

#Transactions for this input:
print("The input transactions for this dataset are: ",'\n')
for i in listoflist_dataset:
    print(i)
print('\n')

# Print results
print("Frequent Itemsets (FP-growth):")
print(frequent_itemsets_fp)

print("\nAssociation Rules (FP-growth):")
print(rules_fp)

Enter minimum Support: 0.15
Enter minimum Confidence: 0.7
The input transactions for this dataset are:  

["A Beginner's Guide", ' Java: The Complete Reference', ' Java For Dummies', ' Android Programming: The Big Nerd Ranch']
["A Beginner's Guide", ' Java: The Complete Reference', ' Java For Dummies']
["A Beginner's Guide", ' Java: The Complete Reference', ' Java For Dummies', ' Android Programming: The Big Nerd Ranch', ' Head First Java 2nd Edition']
['Android Programming: The Big Nerd Ranch', ' Head First Java 2nd Edition ', ' Beginning Programming with Java']
['Android Programming: The Big Nerd Ranch', ' Beginning Programming with Java', ' Java 8 Pocket Guide']
["A Beginner's Guide", ' Android Programming: The Big Nerd Ranch', ' Head First Java 2nd Edition']
["A Beginner's Guide", ' Head First Java 2nd Edition ', ' Beginning Programming with Java']
['Java: The Complete Reference', ' Java For Dummies', ' Android Programming: The Big Nerd Ranch']
['Java For Dummies', ' Android Progra